In [1]:
from flash.image.classification.integrations.baal import (
    ActiveLearningDataModule,
)
from ndropALLoop import ndrop_ActiveLearningLoop
from flash.image import ImageClassifier, ImageClassificationData

from baal.active.dataset import ActiveLearningDataset
from baal.active import get_heuristic

import os

import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torchmetrics import Accuracy

from torchvision import datasets
from torchvision.transforms import transforms

In [8]:
from pl_bolts.datamodules import CIFAR10DataModule

In [13]:
cdm = CIFAR10DataModule(data_dir = "./data")
cdm.setup()

/home/betairya/anaconda3/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:115: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."
/home/betairya/anaconda3/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:134: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7."
/home/betairya/anaconda3/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:153: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7."


In [14]:
cdm.val_dataloader().dataset

In [2]:
IMG_SIZE = 32
train_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)
test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

class DataModule_(ImageClassificationData):
    @property
    def num_classes(self):
        return 10

def get_data_module(heuristic, data_path):
    train_set = datasets.CIFAR10(data_path, train=True, download=True)
    test_set = datasets.CIFAR10(data_path, train=False, download=True)
    dm = DataModule_.from_datasets(
        train_dataset=train_set,
        test_dataset=test_set,
        train_transform=train_transforms,
        test_transform=test_transforms,
        # Do not forget to set `predict_transform`,
        # this is what we will use for uncertainty estimation!
        predict_transform=test_transforms,
        batch_size=64,
    )
    active_dm = ActiveLearningDataModule(
        dm,
        heuristic=get_heuristic(heuristic),
        initial_num_labels=1024,
        query_size=100,
        val_split=0.0,
    )
    assert active_dm.has_test, "No test set?"
    return active_dm

In [3]:
class SimpleModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(32 * 32 * 3, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.accuracy(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [4]:
# Init our model
model = SimpleModel()

active_dm = get_data_module('random', './data')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
aloop = ndrop_ActiveLearningLoop(
    label_epoch_frequency = 10,
    inference_iteration = 1
)

In [6]:
# Initialize a trainer
trainer = Trainer(
    gpus=1,
    max_epochs=10,
    progress_bar_refresh_rate=20,
)

aloop.connect(trainer.fit_loop)
trainer.fit_loop = aloop

/home/betairya/anaconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
# Train the model ⚡
trainer.fit(model, datamodule = active_dm)

/home/betairya/anaconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 30.7 K
--------------------------------
30.7 K    Trainable params
0         Non-trainable params
30.7 K    Total params
0.123     Total estimated model params size (MB)
/home/betairya/anaconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, doe

Training: 0it [00:00, ?it/s]

TypeError: img should be PIL Image. Got <class 'dict'>